In [1]:
import face_rec

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [2]:
import pandas as pd
import numpy as np
import cv2

In [3]:
face_rec.r.ping()

True

In [4]:
### Step 1: extract data from database

In [5]:
name = 'company:register'
retrive_dict = face_rec.r.hgetall(name)
retrive_series = pd.Series(retrive_dict)
retrive_series = retrive_series.apply(lambda x: np.frombuffer(x,dtype=np.float32))
index = retrive_series.index
index = list(map(lambda x: x.decode(), index))
retrive_series.index = index
retrive_df = retrive_series.to_frame().reset_index()
retrive_df.columns = ['name_role','facial_features']
retrive_df[['name','role']] = retrive_df['name_role'].apply(lambda x: x.split('@')).apply(pd.Series)
retrive_df

,name_role,facial_features,name,role
0,abc@Student,"[0.1912692, -0.48850536, -1.6902622, 0.4422732...",abc,Student
1,Scarlett Johansson@Student,"[0.1912692, -0.48850536, -1.6902622, 0.4422732...",Scarlett Johansson,Student
2,Annabeth@Student,"[0.1912692, -0.48850536, -1.6902622, 0.4422732...",Annabeth,Student
3,Chris Evans@Student,"[0.72002447, -1.4682528, -0.77743137, -1.45999...",Chris Evans,Student
4,percy@Admin,"[0.1912692, -0.48850536, -1.6902622, 0.4422732...",percy,Admin
5,Morgan Freeman@Teacher,"[0.48562637, -0.61965686, -0.4072629, 1.068597...",Morgan Freeman,Teacher
6,Barack Obama@Teacher,"[0.94846946, -0.5583677, 0.09491567, -1.279613...",Barack Obama,Teacher
7,Yuthika@Student,"[0.1912692, -0.48850536, -1.6902622, 0.4422732...",Yuthika,Student
8,qwerty@Teacher,"[0.1912692, -0.48850536, -1.6902622, 0.4422732...",qwerty,Teacher
9,Angelina Jolie@Student,"[0.4324317, 1.074975, -0.12851736, -1.833421, ...",Angelina Jolie,Student


In [6]:
###Step 2: Get real time predictions

In [8]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    
    if ret == False:
        break
        
    pred_frame = face_rec.face_prediction(frame,retrive_df,'facial_features',['name','role'],thresh=0.5)
    
    #cv2.imshow('frame',frame)
    cv2.imshow('prediction',pred_frame)
    
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()
cap.release()                                        